In [1]:
from collections import defaultdict
from tabulate import tabulate
productions = {
    "S'": ["S"],
    "S": ["AA"],
    "A": ["aA", "b"]
}
terminals = ['a', 'b', '$']
non_terminals = ['S', 'A']
symbols = terminals + non_terminals
def closure(items):
    closure_set = set(items)
    while True:
        new_items = set()
        for lhs, rhs, dot in closure_set:
            if dot < len(rhs):
                sym = rhs[dot]
                if sym in productions:
                    for prod in productions[sym]:
                        item = (sym, prod, 0)
                        if item not in closure_set:
                            new_items.add(item)
        if not new_items:
            break
        closure_set.update(new_items)
    return closure_set
def goto(I, symbol):
    moved_items = set()
    for lhs, rhs, dot in I:
        if dot < len(rhs) and rhs[dot] == symbol:
            moved_items.add((lhs, rhs, dot + 1))
    return closure(moved_items)
def items():
    start_item = ("S'", productions["S'"][0], 0)
    C = [closure({start_item})]
    seen = {frozenset(C[0])}
    while True:
        added = False
        for I in C:
            for sym in symbols:
                goto_I = goto(I, sym)
                if goto_I and frozenset(goto_I) not in seen:
                    C.append(goto_I)
                    seen.add(frozenset(goto_I))
                    added = True
        if not added:
            break
    return C
def build_parsing_table(C):
    table = []
    state_indices = {frozenset(I): idx for idx, I in enumerate(C)}

    for i in range(len(C)):
        row = {"State": i}
        for t in terminals + ['$']:
            row[t] = ""
        for nt in non_terminals:
            row[nt] = ""
        table.append(row)
    for idx, I in enumerate(C):
        for item in I:
            lhs, rhs, dot = item
            if dot < len(rhs):
                symbol = rhs[dot]
                goto_I = goto(I, symbol)
                j = state_indices.get(frozenset(goto_I))
                if j is not None:
                    if symbol in terminals:
                        table[idx][symbol] = f"s{j}"
                    elif symbol in non_terminals:
                        table[idx][symbol] = f"{j}"
            else:
                if lhs == "S'":
                    table[idx]['$'] = "acc"
                else:
                    for t in terminals + ['$']:
                        if table[idx][t] == "":
                            table[idx][t] = f"r({lhs}→{rhs})"
    return table
def print_dfa_states(C):
    print("DFA of LR(0) Item Sets:\n")
    for idx, state in enumerate(C):
        print(f"State {idx}:")
        for lhs, rhs, dot in sorted(state):
            before_dot = rhs[:dot]
            after_dot = rhs[dot:]
            print(f"  {lhs} → {before_dot}.{after_dot}")
        print()
def print_parsing_table(table):
    headers = table[0].keys()
    rows = [list(row.values()) for row in table]
    print("\n LR(0) Parsing Table:\n")
    print(tabulate(rows, headers=headers, tablefmt="grid"))
C = items()
print_dfa_states(C)
parsing_table = build_parsing_table(C)
print_parsing_table(parsing_table)

DFA of LR(0) Item Sets:

State 0:
  A → .aA
  A → .b
  S → .AA
  S' → .S

State 1:
  A → .aA
  A → a.A
  A → .b

State 2:
  A → b.

State 3:
  S' → S.

State 4:
  A → .aA
  A → .b
  S → A.A

State 5:
  A → aA.

State 6:
  S → AA.


 LR(0) Parsing Table:

+---------+---------+---------+---------+-----+-----+
|   State | a       | b       | $       | S   | A   |
+=========+=========+=========+=========+=====+=====+
|       0 | s1      | s2      |         | 3   | 4   |
+---------+---------+---------+---------+-----+-----+
|       1 | s1      | s2      |         |     | 5   |
+---------+---------+---------+---------+-----+-----+
|       2 | r(A→b)  | r(A→b)  | r(A→b)  |     |     |
+---------+---------+---------+---------+-----+-----+
|       3 |         |         | acc     |     |     |
+---------+---------+---------+---------+-----+-----+
|       4 | s1      | s2      |         |     | 6   |
+---------+---------+---------+---------+-----+-----+
|       5 | r(A→aA) | r(A→aA) | r(A→aA) |  